# Accessing MAST using Astroquery: An Example

In this example we will:
* Look for a specific HST observation
* Download and view the associated HST image
* Find Gaia and HSC sources which fall within the image footprint
* Overlay the catalog sources on the HSC image

## Imports

### astroquery.mast imports

In [ ]:
from astroquery.mast import Observations
from astroquery.mast import Catalogs

### Other imports

In [ ]:
import numpy as np

from astropy.io import fits
from astropy.table import Table
from astropy.visualization import ImageNormalize, HistEqStretch
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.wcs import WCS

import matplotlib.pyplot as plt
%matplotlib inline 

## Querying for the HST image

In [ ]:
hst_m51_obs = Observations.query_criteria(obs_id='j97c34xgq')
hst_m51_obs

In [ ]:
hst_m51_prods = Observations.get_product_list(hst_m51_obs)
print("Number of data products:", len(hst_m51_prods))

In [ ]:
hst_m51_prods.colnames

In [ ]:
hst_m51_prods['description','productType','productSubGroupDescription','productFilename','size']

In [ ]:
hst_m51_manifest = Observations.download_products(hst_m51_prods,productSubGroupDescription='DRZ')
hst_m51_manifest

## Displaying the HST image

In [ ]:
m51_image = fits.getdata(hst_m51_manifest['Local Path'][0])

In [ ]:
fig = plt.figure(figsize=(12,12))
ax = fig.add_subplot(111)

norm = ImageNormalize(m51_image, stretch=HistEqStretch(m51_image[~np.isnan(m51_image)]))
ax.imshow(m51_image, cmap='gray', norm=norm)

plt.show()

## Querying for catalog sources within this footprint

### Use the HST WCS info to determine our search radii

In [ ]:
hst_wcs = WCS(fits.getheader(hst_m51_manifest['Local Path'][0],1))
hst_wcs

In [ ]:
center_ra,center_dec = hst_wcs.wcs.crval
corner_ra,corner_dec = hst_wcs.wcs_pix2world(0,0,0)
approx_radius = ((corner_ra - center_ra)**2 + (corner_dec - center_dec)**2)**0.5
print(approx_radius,"deg")

In [ ]:
coord = SkyCoord(ra=center_ra, dec=center_dec, unit=(u.degree, u.degree), frame='icrs')
radius = u.Quantity(approx_radius, u.deg)

### Searching for Gaia catalog sources

In [ ]:
gaia_sources = Catalogs.query_region(coord,radius,"Gaia")
print("Number of Gaia sources found:",len(gaia_sources))

In [ ]:
gaia_sources[:3]

### Searching for HSC sources

The HSC is a very dense catalog, so in order to make this search a more resonable size, we will only query a subsection of the footprint.

In [ ]:
hsc_sources = Catalogs.query_region(coord,radius/4,"HSC")
print("Number of HSC sources found:",len(hsc_sources))

In [ ]:
hsc_sources[:3]

## Overlaying catalog sources on the HST image

### Overlaying the Gaia sources

In [ ]:
gaia_x,gaia_y = hst_wcs.wcs_world2pix(gaia_sources['ra'],gaia_sources['dec'],0)

In [ ]:
fig = plt.figure(figsize=(20,20))
ax = fig.add_subplot(111)

ymax,xmax = m51_image.shape
ax.set_xlim(0,xmax)
ax.set_ylim(0,ymax)

norm = ImageNormalize(m51_image, stretch=HistEqStretch(m51_image[~np.isnan(m51_image)]))
ax.imshow(m51_image, cmap='gray', norm=norm)
ax.scatter(gaia_x,gaia_y,s=90, facecolors='none', edgecolors='#c0004c', linewidth=2, alpha=0.75)

plt.show()

### Overlaying the HSC sources